In [ ]:
import os
import pandas as pd
import numpy as np
from scipy.stats import kruskal
from scipy.stats import rankdata
import matplotlib.pyplot as plt
import seaborn as sns
import chardet
import csv
import re
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

INPUT_FOLDER = "filtered_data"
RESULTS_FOLDER = "results"
GRAPHS_FOLDER = os.path.join(RESULTS_FOLDER, "graphs")
for folder in [RESULTS_FOLDER, GRAPHS_FOLDER]:
    os.makedirs(folder, exist_ok=True)

sns.set_style("whitegrid")

def detect_encoding(file_path):
    """Визначає кодування файлу."""
    try:
        with open(file_path, 'rb') as f:
            return chardet.detect(f.read(10000))['encoding'] or 'utf-8'
    except Exception as e:
        print(f"Помилка визначення кодування для {file_path}: {e}")
        return 'utf-8'

def detect_separator(file_path, encoding):
    """Визначає роздільник у CSV-файлі."""
    try:
        with open(file_path, encoding=encoding) as f:
            sample = f.read(2048)
            return csv.Sniffer().sniff(sample).delimiter
    except Exception as e:
        print(f"Помилка визначення роздільника для {file_path}: {e}")
        return ','

def load_csv(file_path):
    """Завантажує CSV-файл."""
    encoding = detect_encoding(file_path)
    sep = detect_separator(file_path, encoding)
    print(f"   Кодування: {encoding}, роздільник: '{sep}'")
    
    try:
        df = pd.read_csv(file_path, encoding=encoding, sep=sep, quotechar='"', on_bad_lines='skip', low_memory=False)
        print(f"   Перші 5 колонок: {list(df.columns[:5])}")
        return df
    except Exception as e:
        print(f"Помилка читання {file_path}: {e}")
        return None

def detect_dataset_type(filename):
    match = re.search(r'(\d{4})', filename)
    if match:
        year = int(match.group(1))
        return 'NMT' if year >= 2022 else 'ZNO'
    return None

def compute_mean_scores(df, filename):
    year = int(filename.split('.')[0])
    df_scores = df[df['average_score'].notna() & df['education_org_type'].notna()].copy()
    mean_scores = df_scores.groupby('education_org_type')['average_score'].mean().reset_index()
    mean_scores_dict = {t: mean_scores[mean_scores['education_org_type'] == t]['average_score'].mean() 
                       if t in mean_scores['education_org_type'].values else np.nan 
                       for t in mean_scores['education_org_type'].unique()}
    mean_scores_pivot = pd.DataFrame([mean_scores_dict])
    mean_scores_pivot['year'] = year
    table_path = os.path.join("results", "mean_scores_all_years.csv")
    if os.path.exists(table_path):
        existing_df = pd.read_csv(table_path)
        existing_df = existing_df[existing_df["year"] != year]
        updated_df = pd.concat([existing_df, mean_scores_pivot], ignore_index=True)
    else:
        updated_df = mean_scores_pivot
    updated_df.to_csv(table_path, index=False)
    print(f"✅ Таблиця середніх балів оновлено: {table_path}")

def cluster_education_types(table_path):
    if os.path.exists(table_path):
        cluster_data = pd.read_csv(table_path)
        if len(cluster_data) >= 2 and cluster_data.drop('year', axis=1).notna().any().any():
            types = [col for col in cluster_data.columns if col != 'year']
            cluster_data_selected = cluster_data[['year'] + types]

            cluster_data_transposed = cluster_data_selected.set_index('year').transpose()
            X = cluster_data_transposed.values
            
            column_means = np.nanmean(X, axis=0)
            X = np.where(np.isnan(X), np.tile(column_means, (X.shape[0], 1)), X)
            scaler = StandardScaler()
            X_scaled = scaler.fit_transform(X)

            pca = PCA(n_components=2)
            X_pca = pca.fit_transform(X_scaled)

            kmeans = KMeans(n_clusters=4, random_state=42)
            kmeans_clusters = kmeans.fit_predict(X_scaled)

            agg_clustering = AgglomerativeClustering(n_clusters=4)
            agg_clusters = agg_clustering.fit_predict(X_scaled)

            plt.figure(figsize=(10, 6))
            scatter_kmeans = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=kmeans_clusters, cmap='viridis', s=100)
            plt.title("Кластеризація типів навчальних закладів (KMeans, 4 кластерів)")
            plt.xlabel("Перша головна компонента")
            plt.ylabel("Друга головна компонента")
            types = cluster_data_transposed.index
            for i, txt in enumerate(types):
                plt.annotate(txt, (X_pca[i, 0], X_pca[i, 1]), xytext=(5, 5), textcoords='offset points')
            plt.colorbar(scatter_kmeans, label='Кластер')
            kmeans_path = os.path.join("results/graphs", "cluster_education_kmeans.png")
            plt.savefig(kmeans_path, dpi=300, bbox_inches="tight")
            plt.close()
            print(f"✅ Графік кластеризації (KMeans) збережено: {kmeans_path}")

            plt.figure(figsize=(10, 6))
            scatter_agg = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=agg_clusters, cmap='plasma', s=100)
            plt.title("Кластеризація типів навчальних закладів (Agglomerative, 4 кластерів)")
            plt.xlabel("Перша головна компонента")
            plt.ylabel("Друга головна компонента")
            for i, txt in enumerate(types):
                plt.annotate(txt, (X_pca[i, 0], X_pca[i, 1]), xytext=(5, 5), textcoords='offset points')
            plt.colorbar(scatter_agg, label='Кластер')
            agg_path = os.path.join("results/graphs", "cluster_education_agg.png")
            plt.savefig(agg_path, dpi=300, bbox_inches="tight")
            plt.close()
            print(f"✅ Графік кластеризації (Agglomerative) збережено: {agg_path}")
        else:
            print("⚠️ Недостатньо даних для кластеризації")
                                                            
for filename in os.listdir(INPUT_FOLDER):
    if not filename.endswith(".csv"):
        continue
    
    file_path = os.path.join(INPUT_FOLDER, filename)
    print(f"🔄 Обробка {filename}...")
    
    try:
        df = load_csv(file_path)
        if df is None or df.empty:
            print(f"⚠️ Файл {filename} порожній або не вдалося завантажити")
            continue
        
        dataset_type = detect_dataset_type(filename)
        if not dataset_type:
            print(f"⚠️ Невідомий тип даних: {filename}")
            continue
        
        print(f"   Завантажено {len(df)} рядків, {len(df.columns)} стовпців")
        compute_mean_scores(df, filename)
    
    except Exception as e:
        print(f"❌ Помилка у файлі {filename}: {e}")
        import traceback
        traceback.print_exc()

table_path = os.path.join("results", "mean_scores_all_years.csv")
cluster_education_types(table_path)

print(f"\n🎉 Аналіз завершено! Результати збережено в '{RESULTS_FOLDER}'")